# 6장 sLLM 학습하기

In [ ]:
# !pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 huggingface_hub==0.22.2 autotrain-advanced==0.7.77 -qqq
# !pip install --upgrade huggingface-hub -qqq

## 성능 평가 파이프라인 준비하기

### SQL 성능 프롬프트

- 예제 6.2. SQL 프롬프트

In [1]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

### GPT-4 평가 프롬프트와 코드 준비

- 예제 6.4. 평가를 위한 요청 jsonl 작성 함수

In [2]:
# 필요한 라이브러리 임포트
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  """
  GPT 평가를 위한 요청 파일을 생성하는 함수
  
  Args:
      df: 평가 데이터가 포함된 데이터프레임
      filename: 저장할 파일 이름 
      dir: 저장할 디렉토리 경로 (기본값: 'requests')
  """
  # 디렉토리가 없으면 생성
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)
      
  # 프롬프트 리스트 생성
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)

  # GPT-4 요청을 위한 job 리스트 생성
  jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  
  # 파일에 job을 jsonl 형식으로 저장
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")

- 예제 6.5. 비동기 요청 명령

In [ ]:
import os
# os.environ["OPENAI_API_KEY"] = "자신의 OpenAI API 키 입력"

python api_request_parallel_processor.py \
  --requests_filepath {요청 파일 경로} \
  --save_filepath {생성할 결과 파일 경로} \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 300 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

- 예제 6.6. 결과 jsonl 파일을 csv로 변환하는 함수

In [3]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

## 실습: 미세 조정 수행하기

### 기초 모델 평가하기

In [4]:
# PyTorch와 Transformers 라이브러리 임포트
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# 추론 파이프라인 생성 함수 정의
def make_inference_pipeline(model_id):
  # 토크나이저 로드
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  # 4비트 양자화를 적용한 모델 로드 
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
  # 텍스트 생성 파이프라인 생성
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

# Yi-Ko-6B 모델 ID 설정
model_id = 'beomi/Yi-Ko-6B'
# 추론 파이프라인 생성
hf_pipe = make_inference_pipeline(model_id)

/home/restful3/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


In [6]:
# SQL 생성을 위한 예제 프롬프트
example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""

# 모델을 사용하여 SQL 쿼리 생성
hf_pipe(example, do_sample=False,
    return_full_text=False, max_length=512, truncation=True)
#  SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇의 결과:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%'; (생략)

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE 

- 예제 6.8. 기초 모델 성능 측정

In [ ]:
# 데이터셋 라이브러리 임포트
from datasets import load_dataset

# ko_text2sql 데이터셋의 테스트 세트 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
# pandas DataFrame으로 변환
df = df.to_pandas()

# 각 데이터에 대해 프롬프트 생성
for idx, row in df.iterrows():
  # context와 question을 조합하여 프롬프트 생성
  prompt = make_prompt(row['context'], row['question'])
  # 생성된 프롬프트를 DataFrame에 저장
  df.loc[idx, 'prompt'] = prompt

# 모델을 사용하여 SQL 쿼리 생성
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
                   return_full_text=False, max_length=512, truncation=True)
# 생성된 SQL 쿼리 추출
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
# 생성된 SQL을 DataFrame에 저장
df['gen_sql'] = gen_sqls

# GPT 평가를 위한 JSONL 파일 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)